# 3. Building Prediction Model

### 텐서플로우와 RandomForest를 사용하여 모델을 만들어 봤습니다

### Tensorflow

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf

In [4]:
x = pd.read_csv('X_train')

In [5]:
len(x.columns)

90

In [6]:
tf.reset_default_graph()

In [ ]:
filename_queue = tf.train.string_input_producer(
['X_train'], shuffle=False, name='filename_queue')

reader = tf.TextLineReader()
key, value = reader.read(filename_queue)

record_defaults = [[0.] for cl in range(103)]
xy = tf.decode_csv(value, record_defaults = record_defaults)
train_x_batch, train_y_batch = tf.train.batch([xy[0:-2], xy[-2:]], batch_size=100)
learning_rate = 0.001


X = tf.placeholder(tf.float32, [None, 101], name = 'x-input')
Y = tf.placeholder(tf.float32, [None, 2], name = 'y-input')

keep_prob = tf.placeholder(tf.float32)



W1 = tf.get_variable("weight1", shape=[101, 50],
                     initializer=tf.contrib.layers.xavier_initializer())

b1 = tf.Variable(tf.random_normal([50]), name='bias1')
L1 = tf.nn.relu(tf.matmul(X, W1) + b1)
L1 = tf.nn.dropout(L1, keep_prob=0.5)


W2 = tf.get_variable("weight2", shape = [50, 70],
                     initializer=tf.contrib.layers.xavier_initializer())
b2 = tf.Variable(tf.random_normal([70]), name='bias2')
L2 = tf.nn.relu(tf.matmul(L1, W2) + b2)
    
L2 = tf.nn.dropout(L2, keep_prob=0.5)


W3 = tf.get_variable("weight3", shape = [70, 30],
                     initializer=tf.contrib.layers.xavier_initializer())
b3 = tf.Variable(tf.random_normal([30]), name='bias3')
L3 = tf.nn.relu(tf.matmul(L2, W3) + b3)
    
L3 = tf.nn.dropout(L3, keep_prob=0.5)



W4 = tf.get_variable("weight4", shape = [30, 15],
                     initializer=tf.contrib.layers.xavier_initializer())
b4 = tf.Variable(tf.random_normal([15]), name='bias4')
L4 = tf.nn.relu(tf.matmul(L3, W4) + b4)
    
L4 = tf.nn.dropout(L4, keep_prob=0.5)


W5 = tf.get_variable("weight5", shape = [15, 10],
                     initializer=tf.contrib.layers.xavier_initializer())
b5 = tf.Variable(tf.random_normal([10]), name='bias5')
L5 = tf.nn.relu(tf.matmul(L4, W5) + b5)
    
L5 = tf.nn.dropout(L5, keep_prob=0.5)


W6 = tf.get_variable("weight6", shape = [10, 20],
                     initializer=tf.contrib.layers.xavier_initializer())
b6 = tf.Variable(tf.random_normal([20]), name='bias6')
L6 = tf.nn.relu(tf.matmul(L5, W6) + b6)
    
L6 = tf.nn.dropout(L6, keep_prob=0.5)


    

W7 = tf.get_variable("weight7", shape = [20, 10],
                     initializer=tf.contrib.layers.xavier_initializer())
b7 = tf.Variable(tf.random_normal([10]), name='bias7')
L7 = tf.nn.relu(tf.matmul(L6, W7) + b7)
    
L7 = tf.nn.dropout(L7, keep_prob=0.5)
    

W8 = tf.get_variable("weight8", shape = [10, 2],
                     initializer=tf.contrib.layers.xavier_initializer())
b8 = tf.Variable(tf.random_normal([2]), name='bias8')
hypothesis = tf.matmul(L7, W8) + b8



    
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=hypothesis, labels=Y))
train = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

sess= tf.Session()
coord = tf.train.Coordinator()
threads = tf.train.start_queue_runners(sess=sess, coord=coord)


sess.run(tf.global_variables_initializer())

saver = tf.train.Saver()
saver.save(sess, save_path = '/practice/my_model', global_step = 10000)
    
for step in range(3000000):
    x_batch, y_batch = sess.run([train_x_batch, train_y_batch])
    sess.run( train, feed_dict={X: x_batch, Y: y_batch})

    if step % 100 == 0:
        print(step, sess.run(cost, feed_dict={
                X: x_batch, Y: y_batch}))
#coord.request_stop()
#coord.join(threads)


print('Learning Finished!')



In [8]:
X_test = pd.read_csv('X_test')
x_data = X_test.iloc[:, :-2].values
y_data = X_test.iloc[:, -2:].values
y_data = np.reshape(y_data, (-1, 2))

In [11]:
correct_prediction = tf.equal(tf.argmax(hypothesis, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
print('Accuracy:', sess.run(accuracy, feed_dict={
      X: x_data, Y: y_data,  keep_prob: 1.0}))

Accuracy: 0.650303


텐서플로우의 경우 learn rate, batch size, number of layer를 중점으로 모델을 수정하며 평균적으로 1시간 반 정도 학습 시켜보았지만 cost가 0.60 아래로  내려가지 않았습니다. 그 결과 저는 다음과 같은 문제가 있을 거라고 생각 하였습니다

1. input 변수들이 옳바르지 않거나 더 세부적인 필터링이 필요하다
2. 텐서플로우의 모델이 잘못되었다
3. 더 많은 학습 시간이 필요하다

### Random Forest

In [ ]:
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.metrics import confusion_matrix, f1_score, classification_report, precision_recall_curve, roc_auc_score, accuracy_score
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier

In [ ]:
def heatmap(values, xlabel, ylabel, xticklabels, yticklabels, cmap=None,
             vmin=None, vmax=None, ax=None, fmt="%0.2f"):
    
    if ax is None:
        ax = plt.gca()
      # plot the mean cross-validation scores
    img = ax.pcolor(values, cmap=cmap, vmin=vmin, vmax=vmax)
    img.update_scalarmappable()
    ax.set_xlabel(xlabel)
    ax.set_ylabel(ylabel)
    ax.set_xticks(np.arange(len(xticklabels)) + .5)
    ax.set_yticks(np.arange(len(yticklabels)) + .5)
    ax.set_xticklabels(xticklabels)
    ax.set_yticklabels(yticklabels)
    ax.set_aspect(1)
 
    for p, color, value in zip(img.get_paths(), img.get_facecolors(),
                                img.get_array()):
        
        x, y = p.vertices[:-2, :].mean(0)
        if np.mean(color[:3]) > 0.5:
            c = 'k'
        else:
             c = 'w'
        ax.text(x, y, fmt % value, color=c, ha="center", va="center")
    return img

In [ ]:
param_gridRF = {'max_features' : [1, 10 ,100],
                'min_samples_leaf' : [1, 10, 100]}
grid_searchRF = GridSearchCV(RandomForestClassifier(n_jobs = -1), param_gridRF, cv = 10)
grid_searchRF.fit(X_train[:, -1], y_train)
print(format(grid_searchRF.best_estimator_))

In [ ]:
resultsRF = pd.DataFrame(grid_searchRF.cv_results_)
scoresRF = np.array(resultsRF.mean_test_score).reshape(3,3)
%matplotlib inline
heatmap(scoresRF, xlabel='max_features', xticklabels=param_gridRF['max_features'], ylabel='min_samples_leaf', 
        yticklabels=param_gridRF['min_samples_leaf'], cmap="viridis")

In [ ]:
clf_RF = RandomForestClassifier(n_jobs = -1, man_features = 10, min_samples_leaf = 10)
clf_RF.fit(X_train[:, -1], y_train)
print('Rf accuracy:', clf_RF.score(X_test, y_test))

Random Forest를 그리드 서치를 통해 max_geatures 과 min_sample_leaf를 변환시켜 학습 시켜 본 결과 각 각 10의 값을 가질 떄 가장 높은 정확도를 가진다는 것을 알 수 있습니다. 평균적으로 69%의 정확도가 나오고 kaggle의 리더보드에서 1등을 한 사람의 모델이 71%의 정확도를 보인다는 것을 볼 때 딥러닝 모델보다 짧은 시간 안에(대략 20분) 더 낳은 결과를 예측 하다는 것에 있어서 매우 만족스러운 모델 이라고 할 수 있습니다